# Optimization in MeTTa

Grammar-guided genetic programming (GGGP, G3P) can evolve programs in any user-defined context-free language.

[alogos](https://github.com/robert-haas/alogos) is an ongoing implementation of some G3P methods in Python, supported by DeepFunding.

## Preparation

- MeTTa: I've copied `common.py` from `hyperon_experimental/python/tests/` to `hyperon_experimental/python/hyperon`

- alogos: Installation with `pip install git+https://github.com/robert-haas/alogos.git`

In [1]:
import alogos as al
from hyperon.common import MeTTa

## A function in two variables

In [2]:
metta = MeTTa()
metta.add_parse('(= (func $x $y) (+ (* $x $x) (* $y $y)) )')
metta.interpret('(func 2 3)')

[13]

## Search space

A context-free grammar is used to define a formal language (=a set of strings), which acts as search space.

In [3]:
ebnf_text = """
EXPR = "(func " NUM " " NUM ")"
NUM = DIGIT "." DIGIT DIGIT
DIGIT = "0" | "1" | "2" | "3" | "4" | "5" | "6" | "7" | "8" | "9"
"""

grammar = al.Grammar(ebnf_text=ebnf_text)

Generate a random string

In [4]:
grammar.generate_string()

'(func 9.32 5.34)'

## Objective

An objective function gets a candidate string as input and needs to return a score or fitness for it.

In [5]:
def objective_function(string):
    if len(string) > 100:
        return None  # prevent too long expressions

    result = metta.interpret(string)
    z = result[0].get_object().value
    return z

In [6]:
s = grammar.generate_string()
print(s)

objective_function(s)

(func 3.76 7.68)


73.12

## Optimization

An evolutionary algorithm starts from a population of random strings and improves them one generation after another. It introduces random variations (crossover, mutation) and selects for better individuals (parent selection, survivor selection). At the end it returns the best individual found so far.

In [7]:
num_gen = 50
num_ind = 50

ea = al.EvolutionaryAlgorithm(
    grammar, objective_function, 'min', verbose=True,
    max_or_min_fitness=0.0, max_generations=num_gen, population_size=num_ind, offspring_size=num_ind
)

In [8]:
best_ind = ea.run()

Progress         Generations      Evaluations      Runtime (sec)    Best fitness    
..... .....      10               485              1.6              0.0181
..... ...

Finished         18               858              3.0              0.0             


In [9]:
best_ind

CFG-GP-ST individual:
- Genotype: ((0,3,1,2,7,6,2,7,2,7,4,1,2,7,6,2,7,2,7,5),(5,0,4,1,0,0,1,0,1,0,0,4,1,0,0,1,0,1,0,0))
- Phenotype: (func 0.00 0.00)
- Fitness: 0.0

In [10]:
metta.interpret(best_ind.phenotype)

[0.0]